In [1]:
import pandas as pd 

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from scipy.stats import uniform

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config
set_config(display='diagram')
import datetime
import time

In [2]:
data = pd.read_parquet('../../dataset/processed/artigos_tratados/bertimbau/artigos_tratados_bert_lg.parquet')

In [3]:
df_index = pd.DataFrame({
    'name': [],
    'value':[]
})

In [4]:
linhas_sem_texto = data[data['Conteudo'] == '']
linhas_sem_texto.index 

Index([ 1140,  1165,  1188,  1314,  1320,  1339,  1361,  2003,  2047,  2168,
       ...
       10931, 10933, 10934, 10935, 10936, 10940, 10943, 10944, 10945, 10951],
      dtype='int64', length=180)

In [5]:
data = data[data['Conteudo'] != '']

In [6]:
row_linhas_sem_texto = {
    'name': 'linhas_sem_texto',
    'value': str(linhas_sem_texto.index.to_list())
}


In [7]:
df_index.loc[len(df_index)] = row_linhas_sem_texto

In [8]:
X_columns = [column for column in data.columns if column != 'Vies']
X = data[X_columns] # features
y = data['Vies'] # label


In [9]:
X_train_val_strat_vies, X_test_strat_vies, y_train_val_strat_vies, y_test_strat_vies = train_test_split(X, y,
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=y)

X_train_strat_vies, X_val_strat_vies, y_train_strat_vies, y_val_strat_vies = train_test_split(X_train_val_strat_vies, y_train_val_strat_vies,
                                                                                                test_size=0.1,
                                                                                                random_state=42,
                                                                                                stratify=y_train_val_strat_vies)

In [10]:
strat_vies = {
    'name': 'strat_vies',
    'value': [
        X_train_strat_vies.index.to_list(), 
        X_val_strat_vies.index.to_list(), 
        X_test_strat_vies.index.to_list()
    ]
}
df_index.loc[len(df_index)] = strat_vies

In [11]:
X_train_val_strat_part, X_test_strat_part, y_train_val_strat_part, y_test_strat_part = train_test_split(X, y,
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=data['Partido'])

X_train_strat_part, X_val_strat_part, y_train_strat_part, y_val_strat_part = train_test_split(X_train_val_strat_part, y_train_val_strat_part,
                                                                                                test_size=0.1,
                                                                                                random_state=42,
                                                                                                stratify=X_train_val_strat_part['Partido'])

In [12]:
strat_part = {
    'name': 'strat_part',
    'value': [
        X_train_strat_part.index.to_list(), 
        X_val_strat_part.index.to_list(), 
        X_test_strat_part.index.to_list()
    ]
}

df_index.loc[len(df_index)] = strat_part

In [13]:
part_teste = ['PSTU', 'PV', 'Novo'] # partidos do conjunto de teste

test = data[data['Partido'].isin(part_teste)].copy() # selecao dos dados de teste
test.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

train = data[~data['Partido'].isin(part_teste)].copy() # selecao dos dados de treino
train.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

In [14]:
X_train_val_part_novos = train.drop('Vies', axis=1) # X_train
y_train_val_part_novos = train['Vies'] # y_train

X_train_part_novos, X_val_part_novos, y_train_part_novos, y_val_part_novos = train_test_split(X_train_val_part_novos, y_train_val_part_novos,
                                                                                                test_size=0.1,
                                                                                                random_state=42)

X_test_part_novos = test.drop('Vies', axis=1) # X_test
y_test_part_novos = test['Vies'] # y_test

In [20]:
y_test_part_novos.value_counts()

Vies
direita     1161
centro       814
esquerda     622
Name: count, dtype: int64

In [15]:
part_novos = {
    'name': 'part_novos',
    'value': [
        X_train_part_novos.index.to_list(), 
        X_val_part_novos.index.to_list(), 
        X_test_part_novos.index.to_list()
    ]
}

df_index.loc[len(df_index)] = part_novos

In [16]:
df_index.head()

,name,value
0,linhas_sem_texto,"[1140, 1165, 1188, 1314, 1320, 1339, 1361, 200..."
1,strat_vies,"[[2596, 1763, 6620, 10805, 3029, 2637, 744, 41..."
2,strat_part,"[[4465, 2218, 972, 580, 1957, 5448, 6819, 8621..."
3,part_novos,"[[5497, 6570, 4197, 1214, 5300, 2491, 5284, 38..."


In [17]:
df_index.to_csv('../../dataset/index_splits.csv', index = True)